<a href="https://colab.research.google.com/github/ds-praveenkumar/truefoundry-projects/blob/dev/feedback_prize_effectiveness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install -q kaggle
!pip install -–upgrade pip 
!pip install transformers -q


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -–


In [33]:
!pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [34]:
# from google.colab import files
# files.upload()

In [35]:
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [36]:
!kaggle competitions download -c feedback-prize-effectiveness

feedback-prize-effectiveness.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip feedback-prize-effectiveness.zip -d data

In [39]:
 !pip install mlfoundry -q

In [69]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import mlfoundry
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from tabulate import tabulate
from tqdm import trange
import random

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
mlfoundry.login()

API key is already configured.
Please use `mlfoundry login --relogin` or `mlfoundry.login(relogin=True)`to force relogin


False

In [42]:
client = mlfoundry.get_client()
run = client.create_run(project_name="kaggle", run_name="feedback-prize-effectiveness")


Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/182/e3d5bfbb6c6444e59a3cdf9d54ab9db7/
[mlfoundry] 2022-07-08T05:33:43+0000 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-07-08T05:33:43+0000 INFO Run 'prokaggler/kaggle/feedback-prize-effectiveness-2' has started.


In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print('device selected -> ', device)

device selected ->  cpu


In [73]:
class Config:
  model_path = 'bert-base-uncased'
  max_len = 128
  train_csv = '/content/data/train.csv'
  num_labels = 3

In [53]:
train_df=pd.read_csv(Config.train_csv)
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


In [55]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W+\s*',' ',text)
    text = text.replace('_',' ')
    text = re.sub(r'\d+',' ',text)
    text = re.sub(r'\s\s+',' ',text)
    stops =  list(set(stopwords.words('english'))) 
    stops.extend(list(string.punctuation))
    text = [word.strip() for word in text.split() if word.strip() not in stops]
    
    return ' '.join(text)

In [60]:
train_df['all_text'] = train_df['discourse_text']+' '+train_df['discourse_type']
train_df['clean_text'] = train_df['all_text'].apply(lambda x: clean_text(x))

In [61]:
train_df.drop(['discourse_id','essay_id','discourse_text','discourse_type','all_text'],inplace=True,axis=1)

In [62]:
target_map = {"Ineffective":0, "Adequate":1,"Effective":2}
train_df["target"] = train_df["discourse_effectiveness"].map(target_map)
train_df.head()

,discourse_effectiveness,clean_text,target
0,Adequate,hi isaac going writing face mars natural landf...,1
1,Adequate,perspective think face natural landform dont t...,1
2,Adequate,think face natural landform life mars descover...,1
3,Adequate,life mars would know reason think natural land...,1
4,Adequate,people thought face formed alieans thought lif...,1


In [63]:
train_df.drop(['discourse_effectiveness'],inplace=True,axis=1)

In [66]:
x_train,x_test,y_train_,y_test_ = train_test_split(train_df['clean_text'],train_df['target'],random_state=1234,test_size=0.25)

print(x_train.shape,y_train_.shape)
print(x_test.shape,y_test_.shape)

(27573,) (27573,)
(9192,) (9192,)


In [47]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         print(self.labels,type(self.labels))
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [68]:
tokenizer = BertTokenizer.from_pretrained(Config.model_path)

X_train_tokenized = tokenizer(list(x_train.values),add_special_tokens=True,padding='max_length',truncation=True,max_length=Config.max_len,return_attention_mask=True)
X_val_tokenized = tokenizer(list(x_test.values),add_special_tokens=True,padding='max_length',truncation=True,max_length=Config.max_len,return_attention_mask=True)

train_dataset = Dataset(X_train_tokenized, list(y_train_))
val_dataset = Dataset(X_val_tokenized, list(y_test_))

In [70]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    result = classification_report(labels,pred,output_dict=True)
    return result

In [72]:
model = BertForSequenceClassification.from_pretrained(Config.model_path, num_labels=Config.num_labels)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [75]:
from transformers import EarlyStoppingCallback
from transformers import Trainer,TrainingArguments

In [78]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    save_strategy="epoch",
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [79]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27573
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 20682


RestException: ignored